In [ ]:
import urllib.request
import csv
import os

### **Scrapping Dataset**

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
tahun="2021"
bulan="11"
maxtanggal=31 #tanggal paling akhir dari bulan tersebut
daftar=[]

In [ ]:
def getPage(url):
    page =  urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')
    total = len(soup.find_all('a', class_='pagination__item')) - 1
    halaman=[]
    while total > -1:
        hal = soup.find_all('a', class_='pagination__item')[total].get_text()
        #contoh ['Next', '11', '5', '4', '3', '2', '1']  hanya diambil list pada posisi ke dua yaitu 11 sebagai maksimal looping
        halaman.append(hal)
        total = total - 1
    return halaman[1]


In [ ]:
def getJudul(url):
    page =  urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')
    total = len(soup.find_all('h3')) - 1
    while total > -1:
        judul = soup.find_all('h3')[total].get_text()
        #contoh judul \nFakta-fakta Serangan Dini Hari di Mapolres OKI\n jadi harus dihilangkan \n di depan dan belakang
        judul=judul.lstrip()
        judul=judul.rstrip()
        daftar.append(judul)
        total = total - 1
    return daftar

def getJudulSehari(tanggal, bulan, tahun):
    url='https://news.detik.com/indeks?date='+str(bulan)+'%2F'+str(tanggal)+'%2F'+str(tahun)
    total = int(getPage(url)) #konversi string ke integer
    while total > -1:
          #https://news.detik.com/indeks/11?date=06/28/2020
          urlhal="https://news.detik.com/indeks/"+str(total)+"?date="+str(bulan)+"/"+str(tanggal)+"/"+str(tahun)
          print(urlhal)
          getJudul(urlhal)
          total=total-1
    return daftar

In [ ]:
namafile=tahun+bulan+"-news"
print(namafile)
cwd=os.getcwd()
f=open(cwd+r"/"+namafile+".csv","w+")

202111-news


In [ ]:
x=maxtanggal
while x > 0:
    print(getJudulSehari(x, bulan, tahun))
    print("Data terkumpul: "+str(len(daftar)))
    print("========================================================")
    x=x-1


In [ ]:
x=len(daftar)-1
daftar.reverse()
print(daftar)
while 0 < x:
     f.write(str(daftar[x])+"\n")
     x=x-1

['Kata Mantan Dubes RI untuk Ukraina Soal Invasi Rusia', 'Usai 10 Hari, Basarnas Hentikan Pencarian 4 Korban Longsor-Gempa di Pasaman', 'Jakpro: FEO Tawarkan 4 Pertandingan Interasional Selain Formula E', 'Aturan Lengkap Karantina dari LN Sesuai SE Kemenhub Terbaru', 'Ukraina Minta Indonesia Lantang Bersuara Soal Invasi Rusia', 'Kabar Terkini Sirkuit Formula E yang Mulai Diaspal', 'Dubes: Saat Ini Tidak Ada Tempat Aman di Ukraina untuk Berlindung', 'Cak Imin: Loyalitas Pemilih PKB Tertinggi, Makanya Saya Harus Jadi Presiden', 'Peluang RI Setop Invasi Rusia di Mata Guru Besar UI', 'TNI AU Buka Pendaftaran Komcad 2022 Bagi 500 Orang', 'Kata Ridwan Kamil Soal AMPI Jabar Duetkan Dirinya dengan Airlangga', 'Buya Fikri Minta Maaf Salah Gerakan Salat saat Aksi PA 212', "Momen Ridwan Kamil Sebut 'Airlangga Presiden' di Munas AMPI", 'Cak Imin Klaim Posisi PKB Kini di Papan Atas: Kita Partai Islam Terstabil', 'KKB di Papua Terus Berulah Telan Belasan Korban Jiwa di Awal 2022', 'Bali Bebaskan Kar

### **Training Model**

In [ ]:
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import string

import numpy as np
import pandas as pd
import os

# cwd = os.getcwd()

dataset=pd.read_csv("/content/18class-balance267.csv", delimiter = ';', encoding ='latin1', names=['data', 'target'])



data_train, data_test, target_train, target_test = train_test_split\
(dataset.data, dataset.target, test_size=0.2,random_state=109) 


data_train=np.asarray(data_train)
data_test=np.asarray(data_test)

target_train=target_train.to_numpy()
target_test=target_test.to_numpy()

categories = ['SDG0','SDG1','SDG2','SDG3','SDG4','SDG5','SDG6','SDG7','SDG8','SDG9','SDG10','SDG11','SDG12','SDG13','SDG14','SDG15','SDG16','SDG17']


print("data Train :\n",*data_train," type ",type(data_train))
print("data Test :\n",*data_test," type ",type(data_test))

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
tfidf=TfidfVectorizer(ngram_range=(1, 2))

model = make_pipeline(tfidf, MultinomialNB())


# print("train data:", len(data_train), "type", type(data_train))
# print("train target:", len(target_train), "type", type(target_train))
# print("test data:", len(data_test), "type", type(data_test))

model.fit(data_train, target_train)
labels = model.predict(data_test)
# print("train data:",*data_train," type ",type(data_train))
# print("train target:",*target_train," type ",type(target_train))
# print("model:",*model," type ",type(model))
# print("labels:",*labels," type ",type(labels))

from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

mat = confusion_matrix(target_test, labels)
print(mat)

data Train :
 Melihat Kreasi Tenun dari Desa Siem Aceh Besar 17 Paus Pilot Terdampar di Pulau Sabu NTT 7 Ekor Mati WNA Pekerja Proyek Kereta Cepat Tewas dalam Kebakaran Pipa Pertamina Pameran Buku Frankfurt Komite Buku Nasional Dorong Alih Wahana Konten Jelang Pelantikan Jalan di Depan DPR Masih Ditutup Anies Janji Tak Rusak Cagar Budaya di Kampung Akuarium Derita Penyakit Jantung Sejak Lahir Bayi Kamila Butuh Uluran Tangan Jika Permenkes 30 Diberlakukan RSUD Iskak Tak Mampu Layani Pasien Gagal Ginjal Mulai 2020 Kapal di Perairan RI Wajib Pakai Bahan Bakar Low Sulfur Menjelajah Pulau Senoa yang Memesona Rutan Perempuan Surabaya akan Mulai Dihuni Awal 2020 Mengenal Pulau Laut Perbatasan RI yang Sudah Lepas dari Kegelapan Jokowi Masuk '50 Muslim Berpengaruh Dunia' PKS: Tak Hapus PR dalam Negeri Cerita Tenaga Kesehatan di Miangas Kurang Stok Makanan Hingga Sinyal Menggerakkan Wisata Warisan Industri Kaki Siswi SMP Penderita Kanker Tulang di Mojokerto Telah Diamputasi Sampah Kembali Pemkab

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                ('multinomialnb', MultinomialNB())])

In [ ]:
import pickle
filename = 'sdgs_model.sav'
pickle.dump(model, open(filename, 'wb'))



---



---



## **Load Data and Model**

In [ ]:
import os
import pandas as pd
import pickle
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/Shareddrives/penelitian_data/sdgs' #Letak data di akun gdrive, apabila menggunakan local, ganti path dengan file lokal

In [ ]:
os.listdir(path)

['sdgs_classification.ipynb', 'sdgs_model.sav', '18class-balance267.csv']

In [ ]:
loaded_model_sdgs = pickle.load(open(path+'/sdgs_model.sav', 'rb'))
df = pd.read_csv(path+'/18class-balance267.csv',delimiter = ';', encoding ='latin1', names=['article', 'class'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4806 entries, 0 to 4805
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   article  4806 non-null   object
 1   class    4806 non-null   object
dtypes: object(2)
memory usage: 75.2+ KB


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["article"],df["class"],test_size=.2,random_state=109)

In [ ]:
prediction = loaded_model_sdgs.predict(X_test)
print(metrics.classification_report(y_test,prediction))

              precision    recall  f1-score   support

        SDG0       0.88      0.38      0.53        58
        SDG1       0.66      1.00      0.80        51
       SDG10       0.86      0.91      0.88        55
       SDG11       0.73      0.40      0.52        60
       SDG12       0.80      1.00      0.89        41
       SDG13       0.72      0.93      0.81        44
       SDG14       0.92      1.00      0.96        60
       SDG15       0.87      0.78      0.82        60
       SDG16       0.80      0.69      0.74        52
       SDG17       0.83      0.96      0.89        50
        SDG2       0.83      1.00      0.91        54
        SDG3       0.87      0.90      0.88        50
        SDG4       0.80      0.78      0.79        60
        SDG5       0.84      1.00      0.92        54
        SDG6       0.82      1.00      0.90        51
        SDG7       0.83      1.00      0.90        52
        SDG8       0.89      0.60      0.72        53
        SDG9       0.84    